In [18]:
! pip install tensorflow google-cloud-aiplatform kfp
! pip install google-cloud-storage explainable_ai_sdk 


[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
ERROR: Could not find a version that satisfies the requirement PIL (from versions: none)

[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
ERROR: No matching distribution found for PIL


In [2]:
from google.cloud import aiplatform

aiplatform.init(
    project="minist-microservice",
    location="us-central1",
    staging_bucket="gs://mnist_model_bucket"
)

In [3]:
import tensorflow as tf

# Load MNIST dataset
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

# Filter only digits 3 and 7
train_filter = (y_train == 3) | (y_train == 7)
test_filter = (y_test == 3) | (y_test == 7)

x_train, y_train = x_train[train_filter], y_train[train_filter]
x_test, y_test = x_test[test_filter], y_test[test_filter]

# Normalize pixel values to [0, 1]
x_train = x_train / 255.0
x_test = x_test / 255.0

# Reshape data for CNN input (28x28 images with 1 channel)
x_train = x_train.reshape(-1, 28, 28, 1)
x_test = x_test.reshape(-1, 28, 28, 1)

# Convert labels to binary (3 → 0, 7 → 1)
y_train = (y_train == 7).astype(int)
y_test = (y_test == 7).astype(int)

2025-03-16 22:32:27.231887: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1742164347.260343   17427 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1742164347.268516   17427 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1742164347.291128   17427 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1742164347.291174   17427 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1742164347.291177   17427 computation_placer.cc:177] computation placer alr

In [4]:
from tensorflow.keras import layers, models

model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(1, activation='sigmoid')  # Binary classification
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

/opt/conda/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
2025-03-16 22:32:30.766542: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


In [5]:
history = model.fit(
    x_train, y_train,
    epochs=10,
    batch_size=32,
    validation_data=(x_test, y_test)
)

Epoch 1/10
388/388 ━━━━━━━━━━━━━━━━━━━━ 8s 17ms/step - accuracy: 0.9641 - loss: 0.1021 - val_accuracy: 0.9931 - val_loss: 0.0148
Epoch 2/10
388/388 ━━━━━━━━━━━━━━━━━━━━ 7s 17ms/step - accuracy: 0.9962 - loss: 0.0127 - val_accuracy: 0.9946 - val_loss: 0.0179
Epoch 3/10
388/388 ━━━━━━━━━━━━━━━━━━━━ 6s 16ms/step - accuracy: 0.9976 - loss: 0.0099 - val_accuracy: 0.9971 - val_loss: 0.0058
Epoch 4/10
388/388 ━━━━━━━━━━━━━━━━━━━━ 7s 17ms/step - accuracy: 0.9982 - loss: 0.0068 - val_accuracy: 0.9975 - val_loss: 0.0069
Epoch 5/10
388/388 ━━━━━━━━━━━━━━━━━━━━ 7s 19ms/step - accuracy: 0.9977 - loss: 0.0095 - val_accuracy: 0.9990 - val_loss: 0.0046
Epoch 6/10
388/388 ━━━━━━━━━━━━━━━━━━━━ 7s 18ms/step - accuracy: 0.9982 - loss: 0.0049 - val_accuracy: 0.9971 - val_loss: 0.0089
Epoch 7/10
388/388 ━━━━━━━━━━━━━━━━━━━━ 6s 16ms/step - accuracy: 0.9988 - loss: 0.0033 - val_accuracy: 0.9990 - val_loss: 0.0037
Epoch 8/10
388/388 ━━━━━━━━━━━━━━━━━━━━ 6s 16ms/step - accuracy: 0.9998 - loss: 6.9478e-04 - val_

In [6]:
test_loss, test_acc = model.evaluate(x_test, y_test)
print(f"Test Accuracy: {test_acc:.4f}")

64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.9912 - loss: 0.0296
Test Accuracy: 0.9941


In [7]:
from sklearn.metrics import confusion_matrix
import numpy as np

y_pred = (model.predict(x_test) > 0.5).astype(int)
conf_matrix = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(conf_matrix)

64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
Confusion Matrix:
[[1010    0]
 [  12 1016]]


In [8]:
model.save("mnist_cnn_model.keras")
# Export the model in SavedModel format
model.export("mnist_cnn_model")

INFO:tensorflow:Assets written to: mnist_cnn_model/assets


INFO:tensorflow:Assets written to: mnist_cnn_model/assets


Saved artifact at 'mnist_cnn_model'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 28, 28, 1), dtype=tf.float32, name='keras_tensor')
Output Type:
  TensorSpec(shape=(None, 1), dtype=tf.float32, name=None)
Captures:
  140495408188752: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140495408193680: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140495408198784: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140495408199488: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140495408201424: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140495408199664: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140495408201248: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140495408402976: TensorSpec(shape=(), dtype=tf.resource, name=None)


In [9]:
import explainable_ai_sdk
from explainable_ai_sdk.metadata.tf.v2 import SavedModelMetadataBuilder

# Point to your SavedModel directory
LOCAL_PATH_TO_MODEL = "mnist_cnn_model"  

# Initialize metadata builder
metadata_builder = SavedModelMetadataBuilder(LOCAL_PATH_TO_MODEL)

# Save the model with metadata
LOCAL_PATH_TO_SAVED_MODEL_ARTIFACT = "mnist_cnn_model_with_metadata"
metadata_builder.save_model_with_metadata(LOCAL_PATH_TO_SAVED_MODEL_ARTIFACT)

INFO:tensorflow:Assets written to: mnist_cnn_model_with_metadata/assets


INFO:tensorflow:Assets written to: mnist_cnn_model_with_metadata/assets


'mnist_cnn_model_with_metadata'

In [ ]:
import explainable_ai_sdk

# Load the model with Integrated Gradients configuration
model_with_metadata = explainable_ai_sdk.load_model_from_local_path(
    "mnist_cnn_model_with_metadata",
    config=explainable_ai_sdk.IntegratedGradientsConfig(
        steps=50,  # Increase steps for smoother attributions (slower)
    )
)

AttributeError: module 'explainable_ai_sdk' has no attribute 'IntegratedGradientsConfig'

In [12]:
# 1. Define preprocessing
def _preprocess(bytes_input):
    decoded = tf.io.decode_png(bytes_input, channels=1)
    decoded = tf.image.convert_image_dtype(decoded, tf.float32)
    resized = tf.image.resize(decoded, size=(28, 28))
    return resized

@tf.function(input_signature=[tf.TensorSpec([None], tf.string)])
def preprocess_fn(bytes_inputs):
    with tf.device("cpu:0"):
        decoded_images = tf.map_fn(_preprocess, bytes_inputs, dtype=tf.float32)
    return {"input_1": decoded_images}  # Key matches input layer name

# 2. Define serving function with correct input name
m_call = tf.function(model.call).get_concrete_function(
    [tf.TensorSpec(shape=[None, 28, 28, 1], dtype=tf.float32, name="input_1")]
)

@tf.function(input_signature=[tf.TensorSpec([None], tf.string)])
def serving_fn(bytes_inputs):
    images = preprocess_fn(bytes_inputs)
    prob = m_call(**images)  # Pass keyword args
    return prob

# Export to SavedModel
export_path = 'gs://mnist_model_bucket/mnist_model'
# 3. Export the model
tf.saved_model.save(
    model,
    export_path,
    signatures={
        'serving_default': serving_fn,
        'xai_preprocess': preprocess_fn,
        'xai_model': m_call
    }
)

Instructions for updating:
Use fn_output_signature instead


/opt/conda/lib/python3.10/site-packages/keras/src/models/functional.py:238: UserWarning: The structure of `inputs` doesn't match the expected structure.
Expected: keras_tensor
Received: inputs=['Tensor(shape=(None, 28, 28, 1))']
  warnings.warn(msg)
Instructions for updating:
Use fn_output_signature instead


INFO:tensorflow:Assets written to: gs://mnist_model_bucket/mnist_model/assets


INFO:tensorflow:Assets written to: gs://mnist_model_bucket/mnist_model/assets


In [13]:
from explainable_ai_sdk.metadata.tf.v2 import SavedModelMetadataBuilder

# Initialize metadata builder
builder = SavedModelMetadataBuilder(export_path, signature_name='xai_model')

# Use a black baseline (all zeros for MNIST)
baseline = np.zeros((28, 28, 1)).tolist()  # Grayscale baseline
builder.set_image_metadata(
    input_name='input_1',  # Match model input layer name
    input_baselines=[baseline]
)
builder.save_metadata(export_path)  # Save metadata to the same GCS path

In [16]:
# Create model
MODEL_NAME = "mnist_model"
!gcloud ai-platform models create {MODEL_NAME} --region=us-central1

# Deploy with Integrated Gradients
VERSION_NAME="v1"
! gcloud beta ai-platform versions create $VERSION_NAME \
  --model=$MODEL_NAME \
  --region=us-central1 \
  --framework=TENSORFLOW \
  --runtime-version=2.8 \
  --origin=$export_path \
  --machine-type=n1-standard-4 \
  --explanation-method=integrated-gradients \
  --num-integral-steps=50

Using endpoint [https://us-central1-ml.googleapis.com/]
ERROR: (gcloud.ai-platform.models.create) Resource in projects [minist-microservice] is the subject of a conflict: Field: model.name Error: A model with the same name already exists.
- '@type': type.googleapis.com/google.rpc.BadRequest
  fieldViolations:
  - description: A model with the same name already exists.
    field: model.name
Using endpoint [https://us-central1-ml.googleapis.com/]
Explanations reflect patterns in your model, but don't necessarily reveal fundamental relationships about your data population. See https://cloud.google.com/vertex-ai/docs/explainable-ai/limitations for more information.
ERROR: (gcloud.beta.ai-platform.versions.create) RESOURCE_EXHAUSTED: The requested number of n1-standard-4 exceeds the quota limit. Current usage/limit: 0/0, Requested: 8.


In [24]:
import base64
import io
from PIL import Image

# Convert MNIST test image to base64
test_image = x_test[0].reshape(28, 28, 1) * 255  # Reshape and scale to [0, 255]
test_image = np.squeeze(test_image)  # Remove the extra dimension (28, 28, 1) → (28, 28)
test_image_pil = Image.fromarray(test_image.astype('uint8'), mode='L')
buffer = io.BytesIO()
test_image_pil.save(buffer, format="PNG")
b64_image = base64.b64encode(buffer.getvalue()).decode('utf-8')

# Format instance
instances = [{"bytes_inputs": {"b64": b64_image}}]  # Match serving_fn input name

In [42]:
PROJECT_ID="minist-microservice"

In [43]:
from explainable_ai_sdk import load_model_from_ai_platform

# Load deployed model
remote_model = load_model_from_ai_platform(
    PROJECT_ID,
    MODEL_NAME,
)

# Get explanations
explanations = remote_model.explain(instances)

# Visualize
explanations[0].visualize_attributions()

ValueError: Target URI https://ml.googleapis.com/v1/projects/minist-microservice/models/mnist_model returns HTTP 404 error.
Please check if the project, model, and version names are given correctly.

Updated property [core/project].
